In [11]:

import datetime as dt
import requests
import pandas as pd
import numpy as np
import matplotlib
import mplfinance as mpf
import talib

In [12]:

def now(lookback): 
    #get time now convert it to unix
    timenow = dt.datetime.utcnow()
    end_time = timenow.replace(tzinfo=dt.timezone.utc).timestamp()
    #get lookback and convert to unix 
    start_time = dt.datetime.utcnow() - dt.timedelta(lookback)
    start_time = start_time.replace(tzinfo=dt.timezone.utc).timestamp() 
    return start_time, end_time

_start_time, _end_time = now(3)

data = requests.get(f"https://ftx.com/api//markets/{'BTC-PERP'}/candles?resolution={tf}&start_time={_start_time}&end_time={_end_time}").json()

data = pd.DataFrame(data['result'])

data['time'] = pd.to_datetime(data['time'], unit='ms')
data.set_index('time', inplace= True)
#print(data.head(5))

In [21]:
def getmarkets(PERP):
    markets = requests.get(f"https://ftx.com/api/markets").json()
    markets = pd.DataFrame(markets['result'])
    markets = markets['name'].tolist()
    markets = [market for market in markets if market.endswith(PERP)]
    return markets

perpnames = getmarkets('PERP')
print(perpnames)

['1INCH-PERP', 'AAVE-PERP', 'ADA-PERP', 'AGLD-PERP', 'ALCX-PERP', 'ALGO-PERP', 'ALICE-PERP', 'ALPHA-PERP', 'ALT-PERP', 'AMPL-PERP', 'ANC-PERP', 'APE-PERP', 'AR-PERP', 'ASD-PERP', 'ATLAS-PERP', 'ATOM-PERP', 'AUDIO-PERP', 'AVAX-PERP', 'AXS-PERP', 'BADGER-PERP', 'BAL-PERP', 'BAND-PERP', 'BAO-PERP', 'BAT-PERP', 'BCH-PERP', 'BIT-PERP', 'BNB-PERP', 'BNT-PERP', 'BOBA-PERP', 'BRZ-PERP', 'BSV-PERP', 'BTC-PERP', 'BTT-PERP', 'C98-PERP', 'CAKE-PERP', 'CEL-PERP', 'CELO-PERP', 'CHR-PERP', 'CHZ-PERP', 'CLV-PERP', 'COMP-PERP', 'CONV-PERP', 'CREAM-PERP', 'CRO-PERP', 'CRV-PERP', 'CUSDT-PERP', 'CVC-PERP', 'CVX-PERP', 'DASH-PERP', 'DAWN-PERP', 'DEFI-PERP', 'DENT-PERP', 'DODO-PERP', 'DOGE-PERP', 'DOT-PERP', 'DRGN-PERP', 'DYDX-PERP', 'EDEN-PERP', 'EGLD-PERP', 'ENJ-PERP', 'ENS-PERP', 'EOS-PERP', 'ETC-PERP', 'ETH-PERP', 'EXCH-PERP', 'FIDA-PERP', 'FIL-PERP', 'FLM-PERP', 'FLOW-PERP', 'FTM-PERP', 'FTT-PERP', 'GALA-PERP', 'GMT-PERP', 'GRT-PERP', 'HBAR-PERP', 'HNT-PERP', 'HOLY-PERP', 'HOT-PERP', 'HT-PERP', 'HUM-PE

In [22]:
def ohlc(perpnames, tf, daysback):
    _start_time, _end_time = now(daysback) 
    data = requests.get(f"https://ftx.com/api//markets/{perpnames}/candles?resolution={tf}&start_time={_start_time}&end_time={_end_time}").json()
    data = pd.DataFrame(data['result'])
    return data

In [23]:
dfs = []
tf = 86400
daysback = 50
for perp in perpnames: 
    dfs.append(ohlc(perp, tf, daysback))

In [24]:
all = pd.concat(dict(zip(perpnames,dfs)), axis=1)
#closes = all.loc[:,all.columns.get_level_values(1).isin(['close'])]

nameless = pd.concat(dfs, axis=1)
closes = nameless.loc[:,nameless.columns.get_level_values(0).isin(['close'])]
all

1INCH-PERP                                                \
                    startTime          time    open    high     low   close   
0   2022-01-31T00:00:00+00:00  1.643587e+12  1.6584  1.6942  1.5737  1.6819   
1   2022-02-01T00:00:00+00:00  1.643674e+12  1.6819  1.7439  1.6372  1.7050   
2   2022-02-02T00:00:00+00:00  1.643760e+12  1.7050  1.7475  1.6028  1.6297   
3   2022-02-03T00:00:00+00:00  1.643846e+12  1.6297  1.6623  1.5841  1.6608   
4   2022-02-04T00:00:00+00:00  1.643933e+12  1.6607  1.7662  1.6458  1.7642   
5   2022-02-05T00:00:00+00:00  1.644019e+12  1.7642  1.8465  1.7479  1.7860   
6   2022-02-06T00:00:00+00:00  1.644106e+12  1.7860  1.8664  1.7571  1.8594   
7   2022-02-07T00:00:00+00:00  1.644192e+12  1.8594  1.9603  1.7997  1.9385   
8   2022-02-08T00:00:00+00:00  1.644278e+12  1.9385  1.9827  1.7842  1.8591   
9   2022-02-09T00:00:00+00:00  1.644365e+12  1.8591  1.9551  1.8373  1.9242   
10  2022-02-10T00:00:00+00:00  1.644451e+12  1.9242  1.9513  1.7832  1.7987   
11  2022-02-11T00:00:00+00:00  1.644538e+12  1.7984  1.8418  1.6778  1.7144   
12  2022-02-12T00:00:00+00:00  1.644624e+12  1.7144  1.7391  1.6472  1.6912   
13  2022-02-13T00:00:00+00:00  1.644710e+12  1.6912  1.7062  1.6133  1.6561   
14  2022-02-14T00:00:00+00:00  1.644797e+12  1.6561  1.7365  1.6207  1.6963   
15  2022-02-15T00:00:00+00:00  1.644883e+12  1.6963  1.8023  1.6863  1.7986   
16  2022-02-16T00:00:00+00:00  1.644970e+12  1.7986  1.8570  1.7380  1.8030   
17  2022-02-17T00:00:00+00:00  1.645056e+12  1.8030  1.8272  1.6430  1.6579   
18  2022-02-18T00:00:00+00:00  1.645142e+12  1.6579  1.6803  1.5532  1.5836   
19  2022-02-19T00:00:00+00:00  1.645229e+12  1.5836  1.6038  1.5205  1.5825   
20  2022-02-20T00:00:00+00:00  1.645315e+12  1.5826  1.5841  1.4556  1.4850   
21  2022-02-21T00:00:00+00:00  1.645402e+12  1.4850  1.5534  1.3539  1.3640   
22  2022-02-22T00:00:00+00:00  1.645488e+12  1.3640  1.4376  1.3110  1.4211   
23  2022-02-23T00:00:00+00:00  1.645574e+12  1.4211  1.4829  1.3818  1.3844   
24  2022-02-24T00:00:00+00:00  1.645661e+12  1.3844  1.3947  1.1796  1.3404   
25  2022-02-25T00:00:00+00:00  1.645747e+12  1.3404  1.5025  1.3091  1.4504   
26  2022-02-26T00:00:00+00:00  1.645834e+12  1.4504  1.4806  1.4117  1.4353   
27  2022-02-27T00:00:00+00:00  1.645920e+12  1.4353  1.5380  1.3584  1.3838   
28  2022-02-28T00:00:00+00:00  1.646006e+12  1.3838  1.5053  1.3275  1.5025   
29  2022-03-01T00:00:00+00:00  1.646093e+12  1.5026  1.5530  1.4653  1.5216   
30  2022-03-02T00:00:00+00:00  1.646179e+12  1.5216  1.5474  1.4655  1.4975   
31  2022-03-03T00:00:00+00:00  1.646266e+12  1.4975  1.5146  1.4019  1.4368   
32  2022-03-04T00:00:00+00:00  1.646352e+12  1.4368  1.4408  1.3214  1.3473   
33  2022-03-05T00:00:00+00:00  1.646438e+12  1.3473  1.3648  1.2980  1.3559   
34  2022-03-06T00:00:00+00:00  1.646525e+12  1.3559  1.3683  1.2904  1.3023   
35  2022-03-07T00:00:00+00:00  1.646611e+12  1.3022  1.3330  1.2267  1.2717   
36  2022-03-08T00:00:00+00:00  1.646698e+12  1.2717  1.3210  1.2637  1.2961   
37  2022-03-09T00:00:00+00:00  1.646784e+12  1.2961  1.3999  1.2870  1.3755   
38  2022-03-10T00:00:00+00:00  1.646870e+12  1.3755  1.4593  1.2839  1.3736   
39  2022-03-11T00:00:00+00:00  1.646957e+12  1.3736  1.3737  1.3033  1.3176   
40  2022-03-12T00:00:00+00:00  1.647043e+12  1.3176  1.3749  1.3154  1.3317   
41  2022-03-13T00:00:00+00:00  1.647130e+12  1.3317  1.4173  1.2938  1.3075   
42  2022-03-14T00:00:00+00:00  1.647216e+12  1.3075  1.3474  1.2927  1.3333   
43  2022-03-15T00:00:00+00:00  1.647302e+12  1.3333  1.3687  1.2853  1.3519   
44  2022-03-16T00:00:00+00:00  1.647389e+12  1.3519  1.5533  1.3465  1.5091   
45  2022-03-17T00:00:00+00:00  1.647475e+12  1.5091  1.5476  1.4603  1.4826   
46  2022-03-18T00:00:00+00:00  1.647562e+12  1.4826  1.6254  1.4479  1.5879   
47  2022-03-19T00:00:00+00:00  1.647648e+12  1.5879  1.7422  1.5826  1.6345   
48  2022-03-20T00:00:00+00:00  1.647734e+12  1.6345  1.6587  1.5287  

In [25]:
def divs(closes, columns, ob=70, os=30, period=14):
    """Calculates bullish and bearish RSI divergences under oversold or overbought conditions"""

    closes['RSI'] = talib.RSI(closes.iloc[:, columns])
    closes['rolling_rsi_high'] = closes['RSI'].rolling(period).max()
    closes['rolling_rsi_low'] = closes['RSI'].rolling(period).min()
    closes['rolling_closing_high'] = closes.iloc[:, columns].rolling(period).max()
    closes['rolling_closing_low'] = closes.iloc[:, columns].rolling(period).min()


    closes['new_RSI_high'] = np.where(closes['rolling_rsi_high'] > closes['rolling_rsi_high'].shift(), 1, 0)
    closes['new_RSI_low'] = np.where(closes['rolling_rsi_low'] < closes['rolling_rsi_low'].shift(), 1, 0)


    closes['new_closing_high'] = np.where(closes['rolling_closing_high'] > closes['rolling_closing_high'].shift(), 1, 0)
    closes['new_closing_low'] = np.where(closes['rolling_closing_low'] < closes['rolling_closing_low'].shift(), 1, 0)

    bearishrsidiv = np.where((closes['new_closing_high'] == 1) & (closes['new_RSI_high'] == 0) & (closes['RSI'] > ob), 1, 0)
    bullishrsidiv = np.where((closes['new_closing_low'] == 1) & (closes['new_RSI_low'] == 0) & (closes['RSI'] < os), 1, 0)
   # closes.insert(1, 'bearish_rsi_div', bearishrsidiv)
   # closes.insert(2, 'bullish_rsi_div', bullishrsidiv)
    closes['bearish_rsi_div'] = np.where((closes['new_closing_high'] == 1) & (closes['new_RSI_high'] == 0) & (closes['RSI'] > ob), 1, 0)
    closes['bullish_rsi_div'] = np.where((closes['new_closing_low'] == 1) & (closes['new_RSI_low'] == 0) & (closes['RSI'] < os), 1, 0)
    closes = closes.dropna()
    return closes[['RSI', 'bullish_rsi_div', 'bearish_rsi_div']]


In [26]:
closes.iloc[-1:,]


,close,close,close,close,close,close,close,close,close,close,...,close,close,close,close,close,close,close,close,close,close
49,1.5648,161.58,0.90184,1.519,105.5,0.8003,6.4375,0.28945,3414.35,1.1003,...,0.10655,0.19795,192.18,0.8145,3.2715,20450.0,2075.25,168.8,0.04314,0.547


In [27]:
rsidata = [] 
for column in range(179):
   rsidata.append(divs(closes, column))
alle = pd.concat(dict(zip(perpnames,rsidata)), axis=1)
øve = pd.concat(rsidata, axis = 1)
øve
#alle

C:\Users\jonas\AppData\Local\Temp\ipykernel_19788\3790449500.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closes['RSI'] = talib.RSI(closes.iloc[:, columns])
C:\Users\jonas\AppData\Local\Temp\ipykernel_19788\3790449500.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closes['rolling_rsi_high'] = closes['RSI'].rolling(period).max()
C:\Users\jonas\AppData\Local\Temp\ipykernel_19788\3790449500.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

,RSI,bullish_rsi_div,bearish_rsi_div,RSI,bullish_rsi_div,bearish_rsi_div,RSI,bullish_rsi_div,bearish_rsi_div,RSI,...,bearish_rsi_div,RSI,bullish_rsi_div,bearish_rsi_div,RSI,bullish_rsi_div,bearish_rsi_div,RSI,bullish_rsi_div,bearish_rsi_div


In [28]:

def now(lookback): 
    #get time now convert it to unix
    timenow = dt.datetime.utcnow()
    end_time = timenow.replace(tzinfo=dt.timezone.utc).timestamp()
    #get lookback and convert to unix 
    start_time = dt.datetime.utcnow() - dt.timedelta(lookback)
    start_time = start_time.replace(tzinfo=dt.timezone.utc).timestamp() 
    return start_time, end_time


def getmarkets(PERP):
    markets = requests.get(f"https://ftx.com/api/markets").json()
    markets = pd.DataFrame(markets['result'])
    markets = markets['name'].tolist()
    markets = [market for market in markets if market.endswith(PERP)]
    return markets

perpnames = getmarkets('PERP')

def ohlc(perpnames, tf, daysback):
    _start_time, _end_time = now(daysback) 
    data = requests.get(f"https://ftx.com/api//markets/{perpnames}/candles?resolution={tf}&start_time={_start_time}&end_time={_end_time}").json()
    data = pd.DataFrame(data['result'])
    return data

dfs = []
tf = 14400
daysback = 10
for perp in perpnames: 
    dfs.append(ohlc(perp, tf, daysback))


nameless = pd.concat(dfs, axis=1)
closes = nameless.loc[:,nameless.columns.get_level_values(0).isin(['close'])]


def divs(closes, columns, ob=70, os=30, period=14):
    """Calculates bullish and bearish RSI divergences under oversold or overbought conditions"""

    closes['RSI'] = talib.RSI(closes.iloc[:, columns])
    closes['rolling_rsi_high'] = closes['RSI'].rolling(period).max()
    closes['rolling_rsi_low'] = closes['RSI'].rolling(period).min()
    closes['rolling_closing_high'] = closes.iloc[:, columns].rolling(period).max()
    closes['rolling_closing_low'] = closes.iloc[:, columns].rolling(period).min()


    closes['new_RSI_high'] = np.where(closes['rolling_rsi_high'] > closes['rolling_rsi_high'].shift(), 1, 0)
    closes['new_RSI_low'] = np.where(closes['rolling_rsi_low'] < closes['rolling_rsi_low'].shift(), 1, 0)


    closes['new_closing_high'] = np.where(closes['rolling_closing_high'] > closes['rolling_closing_high'].shift(), 1, 0)
    closes['new_closing_low'] = np.where(closes['rolling_closing_low'] < closes['rolling_closing_low'].shift(), 1, 0)

    closes['bearish_rsi_div'] = np.where((closes['new_closing_high'] == 1) & (closes['new_RSI_high'] == 0) & (closes['RSI'] > ob), 1, 0)
    closes['bullish_rsi_div'] = np.where((closes['new_closing_low'] == 1) & (closes['new_RSI_low'] == 0) & (closes['RSI'] < os), 1, 0)
    closes = closes.dropna()
    return closes[['RSI','bullish_rsi_div', 'bearish_rsi_div']]


rsidata = [] 
for column in range(179):
   rsidata.append(divs(closes, column))
#htf = pd.concat(dict(zip(perpnames,rsidata)), axis=1)
træning = pd.concat(rsidata, axis = 1)
#cols = (htf.iloc[-3:] != 0).any()
#websitedata = htf.iloc[-3:][cols[cols].index]

træning

C:\Users\jonas\AppData\Local\Temp\ipykernel_19788\66127292.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closes['RSI'] = talib.RSI(closes.iloc[:, columns])
C:\Users\jonas\AppData\Local\Temp\ipykernel_19788\66127292.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closes['rolling_rsi_high'] = closes['RSI'].rolling(period).max()
C:\Users\jonas\AppData\Local\Temp\ipykernel_19788\66127292.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

,RSI,bullish_rsi_div,bearish_rsi_div,RSI,bullish_rsi_div,bearish_rsi_div,RSI,bullish_rsi_div,bearish_rsi_div,RSI,...,bearish_rsi_div,RSI,bullish_rsi_div,bearish_rsi_div,RSI,bullish_rsi_div,bearish_rsi_div,RSI,bullish_rsi_div,bearish_rsi_div
